In [9]:
import sys
sys.path.append('../../')
sys.path.append('../')
import time
import onn

In [10]:
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.fx import subgraph_rewriter, symbolic_trace
import utils
from torch.fx import Proxy, Graph, GraphModule
from torch.fx.passes.utils.matcher_utils import SubgraphMatcher

In [11]:
from torch.profiler import profile, record_function, ProfilerActivity
import time
import torch._dynamo as dynamo

In [12]:
onn_model = onn.ONN([10 for i in range(10)],32,[200,200,200],0.1)

In [13]:
traced = symbolic_trace(onn_model)
traced.print_readable()

class ONN(torch.nn.Module):
    def forward(self, x):
        # No stacktrace found for following nodes
        offsets = self.offsets
        add = x + offsets;  x = offsets = None
        embeddings_0 = getattr(self.embeddings, "0")(add)
        embeddings_1 = getattr(self.embeddings, "1")(add)
        embeddings_2 = getattr(self.embeddings, "2")(add)
        embeddings_3 = getattr(self.embeddings, "3")(add)
        embeddings_4 = getattr(self.embeddings, "4")(add)
        embeddings_5 = getattr(self.embeddings, "5")(add)
        embeddings_6 = getattr(self.embeddings, "6")(add)
        embeddings_7 = getattr(self.embeddings, "7")(add)
        embeddings_8 = getattr(self.embeddings, "8")(add)
        embeddings_9 = getattr(self.embeddings, "9")(add);  add = None
        flatten = embeddings_0.flatten(start_dim = 1);  embeddings_0 = None
        getitem = embeddings_1[(slice(None, None, None), 0, slice(None, None, None))]
        getitem_1 = embeddings_1[(slice(None, None, None), 1, s

'class ONN(torch.nn.Module):\n    def forward(self, x):\n        # No stacktrace found for following nodes\n        offsets = self.offsets\n        add = x + offsets;  x = offsets = None\n        embeddings_0 = getattr(self.embeddings, "0")(add)\n        embeddings_1 = getattr(self.embeddings, "1")(add)\n        embeddings_2 = getattr(self.embeddings, "2")(add)\n        embeddings_3 = getattr(self.embeddings, "3")(add)\n        embeddings_4 = getattr(self.embeddings, "4")(add)\n        embeddings_5 = getattr(self.embeddings, "5")(add)\n        embeddings_6 = getattr(self.embeddings, "6")(add)\n        embeddings_7 = getattr(self.embeddings, "7")(add)\n        embeddings_8 = getattr(self.embeddings, "8")(add)\n        embeddings_9 = getattr(self.embeddings, "9")(add);  add = None\n        flatten = embeddings_0.flatten(start_dim = 1);  embeddings_0 = None\n        getitem = embeddings_1[(slice(None, None, None), 0, slice(None, None, None))]\n        getitem_1 = embeddings_1[(slice(None,

In [4]:
def gen_pattern_replace_and_matcher_for_ffm(traced,
                                                  redundancy_part_slice,non_redundancy_part_slice,
                                                  embed_node_name,getitem_node_names,num_field,batch = 4096,match_func = None
                                                ):
  from torch.fx.passes.utils.matcher_utils import SubgraphMatcher
  def _match(match,ori,pat):
    return True 
  class PatternClass(torch.nn.Module):
    def __init__(self,num_fields):
      super().__init__()
      self.num_fields = num_fields
      self.embeddings = torch.nn.ModuleList([
              torch.nn.Embedding(1, 1) for _ in range(self.num_fields-1)
          ])
    def field_aware_interaction(self,field_aware_emb_list):
      interaction = []
      for i in range(self.num_fields - 1):
          for j in range(i + 1, self.num_fields):
              v_ij = field_aware_emb_list[j - 1][:, i, :]
              v_ji = field_aware_emb_list[i][:, j, :]
              dot = torch.sum(v_ij * v_ji, dim=1, keepdim=True)
              interaction.append(dot)
      return torch.cat(interaction, dim=1)  
    def forward(self,x):
      field_aware_emb_list = [self.embeddings[i](x) for i in range(self.num_fields-1)]
      return self.field_aware_interaction(field_aware_emb_list)
  pattern = PatternClass(num_field)  
  pattern_trace = symbolic_trace(pattern)
  pattern_graph = pattern_trace.graph
  original_graph = traced.graph
  matcher =  SubgraphMatcher(pattern_graph, match_output=False, match_placeholder=False,
                              remove_overlapping_matches=True)
  _matches = matcher.match(original_graph)
  # 因为在过滤器中做了限制应该只有一个符合要求的
  _matched = _matches[0]
  pattern_env = utils.get_env(pattern_trace)
  node_map = _matched.nodes_map
  embed_names = [f'embeddings_{i}' for i in range(num_field-1)]
  embed_node_list = [node_map[pattern_env[name]] for name in embed_names]
  embed_node_module_list = [utils.get_target_mod(traced,embed_node.target) for embed_node in embed_node_list]
  class ReplacementClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.embeddings = torch.nn.ModuleList(embed_node_module_list)
      self.num_fields = num_field
      self.num_prefix = redundancy_part_slice[1].stop
      self.num_sufix = self.num_fields - self.num_prefix
    def forward(self,x):
      redundancy_part = x[redundancy_part_slice] 
      non_redundancy_part = x[non_redundancy_part_slice] 
      redundancy_ffm_embed = [self.embeddings[i](redundancy_part) for i in range(self.num_fields-1)]
      non_redundancy_ffm_embed = [self.embeddings[i](non_redundancy_part) for i in range(self.num_fields-1)]
      redundancy_interaction = []
      for i in range(self.num_prefix - 1):
          for j in range(i + 1, self.num_prefix):
              v_ij = redundancy_ffm_embed[j - 1][ i, :]
              v_ji = redundancy_ffm_embed[i][ j, :]
              dot = torch.sum(v_ij * v_ji, dim=-1, keepdim=True)
              redundancy_interaction.append(dot)
              
      non_redundancy_interaction = []
      for i in range(self.num_sufix - 1):
          for j in range(i + 1, self.num_sufix):
              v_ij = non_redundancy_ffm_embed[j - 1+self.num_prefix][:, i, :]
              v_ji = non_redundancy_ffm_embed[i+self.num_prefix][:, j, :]
              dot = torch.sum(v_ij * v_ji, dim=-1, keepdim=True)
              non_redundancy_interaction.append(dot)
      mixed_interaction = []
      for i in range(self.num_prefix):
          for j in range(self.num_sufix):
              v_ij = redundancy_ffm_embed[j - 1+self.num_prefix][ i, :]
              v_ji = non_redundancy_ffm_embed[i - 1][ :,j, :]
              dot = torch.sum(v_ij * v_ji, dim=-1, keepdim=True)
              mixed_interaction.append(dot)
      mixed = torch.concat(mixed_interaction,dim = -1)
      non_redundancy = torch.concat(non_redundancy_interaction,dim = -1)
      redundancy = torch.concat(redundancy_interaction,dim = -1).repeat(batch,1)
      return torch.concat([redundancy,non_redundancy,mixed],dim = -1)
  return pattern,ReplacementClass(),_match    

In [5]:
def workload_onn(num_field, prefix,dim = 64,l = [1024,512,256],batch = 4096):
  print(f"now gen workload of ONN with config: dim: {dim}, num_field: {num_field}, prefix: {prefix}")
  onn_model = onn.ONN([100 for i in range(num_field)],dim,l,0.1)

  model_traced_ori = symbolic_trace(onn_model)
  
  onn_model_modify = onn.ONN([100 for i in range(num_field)],dim,l,0.1)
  onn_model_traced_modify = symbolic_trace(onn_model_modify)
  pattern,replace,match = gen_pattern_replace_and_matcher_for_ffm(onn_model_traced_modify,
                                                                      (0,slice(None,prefix,None)),(slice(None,None,None),slice(prefix,None,None)),
                                                                      embed_node_name = "embedding_embedding",
                                                                      getitem_node_names = ["getitem","getitem_1"],num_field=num_field,batch = batch)
  matches = subgraph_rewriter.replace_pattern_with_filters(onn_model_traced_modify, pattern, replace,[match])
  return model_traced_ori,onn_model_traced_modify

In [6]:
def calculate_mean_and_variance_manual(data):
    n = len(data)
    mean = sum(data) / n
    variance = sum((x - mean) ** 2 for x in data) / n
    return mean, variance

In [7]:
def gen_and_test(num_field = 22,prefix = 10, batch = 4096, dim = 32, workload_func = workload_onn,l = [1024,512,256]):
  def run(model):
    t = torch.randint(low=0, high=88, size=(batch,num_field), dtype=torch.long).cuda()
    traced_model = torch.jit.trace(model.cuda(), t)
    compiled_model = torch.compile(traced_model, backend="inductor")
    compiled_model.eval()
    total_time = []
    for i in range(10):
        start_event = torch.cuda.Event(enable_timing=True)
        end_event = torch.cuda.Event(enable_timing=True)  
        start_event.record()

        with torch.no_grad():
          soutput = compiled_model(t)
        end_event.record()
        torch.cuda.synchronize()
        # 计算并打印函数执行所需的时间
        elapsed_time = start_event.elapsed_time(end_event)
        total_time.append(elapsed_time)
    print(calculate_mean_and_variance_manual(total_time[2:]))
  print(f"now gen workload of DFM with config: dim: {dim}, num_field: {num_field}, prefix: {prefix}, batch :{batch}")
  ori, modify = workload_func(num_field,prefix,dim,l,batch = batch)
  run(ori)
  run(modify)
  # return p1, p2

In [ ]:
def gen_and_test(num_field = 22,prefix = 10, batch = 4096, dim = 32, workload_func = workload_onn,l = [1024,512,256]):
  def run(model):
    t = torch.randint(low=0, high=88, size=(batch,num_field), dtype=torch.long).cuda()
    traced_model = torch.jit.trace(model.cuda(), t)
    compiled_model = torch.compile(traced_model, backend="inductor")
    compiled_model.eval()
    total_time = []
    for i in range(10):
        start_event = torch.cuda.Event(enable_timing=True)
        end_event = torch.cuda.Event(enable_timing=True)  
        start_event.record()

        with torch.no_grad():
          soutput = compiled_model(t)
        end_event.record()
        torch.cuda.synchronize()
        # 计算并打印函数执行所需的时间
        elapsed_time = start_event.elapsed_time(end_event)
        total_time.append(elapsed_time)
    print(calculate_mean_and_variance_manual(total_time[2:]))
  print(f"now gen workload of DFM with config: dim: {dim}, num_field: {num_field}, prefix: {prefix}, batch :{batch}")
  ori, modify = workload_func(num_field,prefix,dim,l,batch = batch)
  run(ori)
  run(modify)

In [ ]:
ori, modify = workload_onn(num_field,prefix,dim,l = [1024,512,256],batch = batch)

In [6]:
def genWorkload(num_field = 34 * 5,prefix = 29 * 5, batch = 4096, dim = 64):
  ori_model_name = f'/home/yssun/pytorch-fm/torchfm/model/test_fx/exp/model_repo/onn/onn_{batch}_{num_field}_{prefix}_{dim}_ori.onnx'
  modify_model_name = f'/home/yssun/pytorch-fm/torchfm/model/test_fx/exp/model_repo/onn/onn_{batch}_{num_field}_{prefix}_{dim}_modify.onnx'
  ori, modify = workload_onn(num_field,prefix,dim,l = [1024,512,256],batch = batch)
  torch.onnx.export(ori,               # 模型 being run
                  torch.randint(low=0, high=20, size=(batch,num_field), dtype=torch.long),                  # 模型输入 (or a tuple for multiple inputs)
                  ori_model_name,        # 导出文件的文件名
                  export_params=True, # 如果设置为True，则参数也会被导出。注意某些情况下参数可能无法被导出。
                  opset_version=10,   # ONNX版本
                  do_constant_folding=True,  # 是否执行常量折叠以优化模型
                  input_names = ['input'],   # 输入的名称
                  output_names = ['output'], # 输出的名称
                  )
  torch.onnx.export(modify,               # 模型 being run
                  torch.randint(low=0, high=20, size=(batch,num_field), dtype=torch.long),                  # 模型输入 (or a tuple for multiple inputs)
                  modify_model_name,        # 导出文件的文件名
                  export_params=True, # 如果设置为True，则参数也会被导出。注意某些情况下参数可能无法被导出。
                  opset_version=10,   # ONNX版本
                  do_constant_folding=True,  # 是否执行常量折叠以优化模型
                  input_names = ['input'],   # 输入的名称
                  output_names = ['output'], # 输出的名称
                  )

In [7]:
dims= [32]
batches = [1024,2048,4096]
num_field_and_prefixs = [(34 * 5,29*5),(22 * 5,10 * 5)]

In [8]:
for dim in dims:
  for batch in batches:
    for num_field,prefix in num_field_and_prefixs:
      genWorkload(num_field=num_field,prefix=prefix,batch=batch,dim=dim)

now gen workload of ONN with config: dim: 32, num_field: 170, prefix: 145


KeyboardInterrupt: 

In [9]:
ori, modify = workload_onn(22 *5,10 *5,32,[1024,512,256],batch = 1024)

now gen workload of ONN with config: dim: 32, num_field: 110, prefix: 50


In [10]:
t = torch.randint(low=0, high=88, size=(1024,22 *5), dtype=torch.long).cuda()

In [21]:
traced_model = torch.jit.trace(ori.cuda(), t)
compiled_model = torch.compile(traced_model, backend="inductor")
compiled_model.eval()

/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1024 / 1024 (100.0%)
Greatest absolute difference: 0.12809452414512634 at index (71,) (up to 1e-05 allowed)
Greatest relative difference: 0.23429494558854905 at index (38,) (up to 1e-05 allowed)
  _check_trace(


OptimizedModule(
  (_orig_mod): ONN(
    original_name=ONN
    (embeddings): Module(
      original_name=Module
      (0): Embedding(original_name=Embedding)
      (1): Embedding(original_name=Embedding)
      (2): Embedding(original_name=Embedding)
      (3): Embedding(original_name=Embedding)
      (4): Embedding(original_name=Embedding)
      (5): Embedding(original_name=Embedding)
      (6): Embedding(original_name=Embedding)
      (7): Embedding(original_name=Embedding)
      (8): Embedding(original_name=Embedding)
      (9): Embedding(original_name=Embedding)
      (10): Embedding(original_name=Embedding)
      (11): Embedding(original_name=Embedding)
      (12): Embedding(original_name=Embedding)
      (13): Embedding(original_name=Embedding)
      (14): Embedding(original_name=Embedding)
      (15): Embedding(original_name=Embedding)
      (16): Embedding(original_name=Embedding)
      (17): Embedding(original_name=Embedding)
      (18): Embedding(original_name=Embedding)
     

In [22]:
ori_compiled = compiled_model

In [11]:
import torch.autograd.profiler as profiler


In [24]:
with torch.profiler.profile(
    activities=[
        torch.profiler.ProfilerActivity.CUDA,  # 记录CUDA活动
    ],
    record_shapes=True,  # 记录操作输入和输出的形状
    profile_memory=True,  # 记录内存使用情况
    with_stack=True,  # 记录代码堆栈信息
    on_trace_ready=torch.profiler.tensorboard_trace_handler('./log')  # 输出TensorBoard日志
) as prof:
    # 在profiler上下文管理器内执行模型
    output = ori_compiled(t)
    torch.cuda.synchronize()  # 确保所有CUDA操作都已完成

# 输出分析结果
print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))

STAGE:2025-02-06 21:33:06 265017:265017 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-06 21:33:06 265017:265017 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-06 21:33:06 265017:265017 ActivityProfilerController.cpp:324] Completed Stage: Post Processing
[W collection.cpp:936] Warning: Failed to recover relationship between all profiler and kineto events: 24238 vs. 0  reassociated. (function reassociate)


-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
void at::native::reduce_kernel<512, 1, at::native::R...         0.00%       0.000us         0.00%       0.000us       0.000us      12.551ms        29.04%      12.551ms       2.094us           0 b           0 b           0 b           0 

In [17]:
traced_model = torch.jit.trace(modify.cuda(), t)
compiled_model = torch.compile(traced_model, backend="inductor")
compiled_model.eval()

/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1024 / 1024 (100.0%)
Greatest absolute difference: 0.13400110602378845 at index (996,) (up to 1e-05 allowed)
Greatest relative difference: 0.22258523962326987 at index (131,) (up to 1e-05 allowed)
  _check_trace(


OptimizedModule(
  (_orig_mod): ONN(
    original_name=ONN
    (embeddings): Module(
      original_name=Module
      (0): Embedding(original_name=Embedding)
      (1): Embedding(original_name=Embedding)
      (2): Embedding(original_name=Embedding)
      (3): Embedding(original_name=Embedding)
      (4): Embedding(original_name=Embedding)
      (5): Embedding(original_name=Embedding)
      (6): Embedding(original_name=Embedding)
      (7): Embedding(original_name=Embedding)
      (8): Embedding(original_name=Embedding)
      (9): Embedding(original_name=Embedding)
      (10): Embedding(original_name=Embedding)
      (11): Embedding(original_name=Embedding)
      (12): Embedding(original_name=Embedding)
      (13): Embedding(original_name=Embedding)
      (14): Embedding(original_name=Embedding)
      (15): Embedding(original_name=Embedding)
      (16): Embedding(original_name=Embedding)
      (17): Embedding(original_name=Embedding)
      (18): Embedding(original_name=Embedding)
     

In [19]:
modify_compiled = compiled_model

In [20]:
with torch.profiler.profile(
    activities=[
        torch.profiler.ProfilerActivity.CUDA,  # 记录CUDA活动
    ],
    record_shapes=True,  # 记录操作输入和输出的形状
    profile_memory=True,  # 记录内存使用情况
    with_stack=True,  # 记录代码堆栈信息
    on_trace_ready=torch.profiler.tensorboard_trace_handler('./log')  # 输出TensorBoard日志
) as prof:
    # 在profiler上下文管理器内执行模型
    output = modify_compiled(t)
    torch.cuda.synchronize()  # 确保所有CUDA操作都已完成

# 输出分析结果
print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))

STAGE:2025-02-06 21:26:10 265017:265017 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-06 21:26:11 265017:265017 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-06 21:26:11 265017:265017 ActivityProfilerController.cpp:324] Completed Stage: Post Processing
[W collection.cpp:936] Warning: Failed to recover relationship between all profiler and kineto events: 24676 vs. 0  reassociated. (function reassociate)


-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
void at::native::reduce_kernel<512, 1, at::native::R...         0.00%       0.000us         0.00%       0.000us       0.000us      12.247ms        28.29%      12.247ms       2.043us           0 b           0 b          5995  
void at::native::(anonymous namespace)::indexSelectL...         0.00%       0.000us         0.0

In [27]:
def gen_pattern_replace_and_matcher_for_SingleMLP(traced,
                                                  redundancy_part_slice,non_redundancy_part_slice,
                                                  key_node_name,getitem_node_names,num_field,dim = 32,batch = 4096,match_func = None
                                                ):
  from torch.fx.passes.utils.matcher_utils import SubgraphMatcher
  def _match(match,ori,pat):
    return True 
  env  = utils.get_env(traced)
  target_node = env[key_node_name]
  target_node_mod = utils.get_target_mod(traced,target_node.target)
  shape_info = target_node_mod.weight.data.shape
  class PatternClass(torch.nn.Module):
    def __init__(self,num_fields):
      super().__init__()
      self.num_fields = num_fields
      # input_dim = dim * self.num_fields + int(self.num_fields * (self.num_fields - 1) / 2)
      self.embeddings = torch.nn.ModuleList([
              torch.nn.Embedding(1, 1) for _ in range(self.num_fields)
          ])
      self.mlp = nn.Linear(shape_info[0],shape_info[1])
    def field_aware_interaction(self,field_aware_emb_list):
      interaction = []
      for i in range(self.num_fields - 1):
          for j in range(i + 1, self.num_fields):
              v_ij = field_aware_emb_list[j - 1][:, i, :]
              v_ji = field_aware_emb_list[i][:, j, :]
              dot = torch.sum(v_ij * v_ji, dim=1, keepdim=True)
              interaction.append(dot)
      return torch.cat(interaction, dim=1)  
    def forward(self,x):
      field_aware_emb_list = [self.embeddings[i](x) for i in range(self.num_fields)]
      diag_embedding = field_aware_emb_list[0].flatten(start_dim=1)
      ffm_out = self.field_aware_interaction(field_aware_emb_list[1:])
      dnn_input = torch.cat([diag_embedding, ffm_out], dim=1)
      return self.mlp(dnn_input)
  pattern = PatternClass(num_field)  
  pattern_trace = symbolic_trace(pattern)
  pattern_graph = pattern_trace.graph
  original_graph = traced.graph
  matcher =  SubgraphMatcher(pattern_graph, match_output=False, match_placeholder=False,
                              remove_overlapping_matches=True)
  _matches = matcher.match(original_graph)
  # 因为在过滤器中做了限制应该只有一个符合要求的
  _matched = _matches[0]
  pattern_env = utils.get_env(pattern_trace)
  node_map = _matched.nodes_map
  embed_names = [f'embeddings_{i}' for i in range(num_field)]
  embed_node_list = [node_map[pattern_env[name]] for name in embed_names]
  embed_node_module_list = [utils.get_target_mod(traced,embed_node.target) for embed_node in embed_node_list]
  linear_node = node_map[pattern_env['mlp']]
  linear_node_module = utils.get_target_mod(traced,linear_node.target)
  linear_node_weight = linear_node_module.weight.data
  linear_node_bias = linear_node_module.bias.data  
  
  
  
  class ReplacementClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.embeddings = torch.nn.ModuleList(embed_node_module_list)
      self.embed_dim = self.embeddings[0].weight.data.shape[1]
      self.num_fields = num_field
      self.num_prefix = redundancy_part_slice[1].stop
      self.num_sufix = self.num_fields - self.num_prefix
      self.redency_weight_len = self.embed_dim * redundancy_part_slice[1].stop + self.num_prefix
      redency_weight = linear_node_weight[:,:self.redency_weight_len]
      unredency_weight = linear_node_weight[:,self.redency_weight_len:]
      self.redency_linear = nn.Linear(redency_weight.shape[1],redency_weight.shape[0])
      self.redency_linear.weight.data.copy_(redency_weight)
      self.redency_linear.bias.data.copy_(linear_node_bias)
      self.unredency_linear = nn.Linear(unredency_weight.shape[1],unredency_weight.shape[0],bias=False)
      self.unredency_linear.weight.data.copy_(unredency_weight)
    def field_aware_interaction(self,field_aware_emb_list):
      interaction = []
      for i in range(self.num_fields - 1):
          for j in range(i + 1, self.num_fields):
              v_ij = field_aware_emb_list[j - 1][:, i, :]
              v_ji = field_aware_emb_list[i][:, j, :]
              dot = torch.sum(v_ij * v_ji, dim=1, keepdim=True)
              interaction.append(dot)
      return torch.cat(interaction, dim=1)  
    def forward(self,x):
      field_aware_emb_list = [self.embeddings[i](x) for i in range(self.num_fields)]
      diag_embedding = field_aware_emb_list[0].flatten(start_dim=1)
      ffm_out = self.field_aware_interaction(field_aware_emb_list[1:])
      dnn_input = torch.cat([diag_embedding, ffm_out], dim=1)
      return self.redency_linear(dnn_input[0,:self.redency_weight_len]) + self.unredency_linear(dnn_input[:,self.redency_weight_len:])
      
  return pattern,ReplacementClass(),_match    

In [28]:
def workload_onn(num_field, prefix,dim = 64,l = [1024,512,256],batch = 4096):
  print(f"now gen workload of ONN with config: dim: {dim}, num_field: {num_field}, prefix: {prefix}")
  onn_model = onn.ONN([100 for i in range(num_field)],dim,l,0.1)

  model_traced_ori = symbolic_trace(onn_model)
  
  onn_model_modify = onn.ONN([100 for i in range(num_field)],dim,l,0.1)
  onn_model_traced_modify = symbolic_trace(onn_model_modify)
  pattern,replace,match = gen_pattern_replace_and_matcher_for_SingleMLP(onn_model_traced_modify,
                                                                      (0,slice(None,prefix,None)),(slice(None,None,None),slice(prefix,None,None)),
                                                                      key_node_name = "mlp_mlp_0",
                                                                      getitem_node_names = ["getitem","getitem_1"],num_field=num_field,batch = batch,dim = dim)
  matches = subgraph_rewriter.replace_pattern_with_filters(onn_model_traced_modify, pattern, replace,[match])
  return model_traced_ori,onn_model_traced_modify

In [29]:
def genWorkload(num_field = 34 * 5,prefix = 29 * 5, batch = 4096, dim = 64):
  ori_model_name = f'/home/yssun/pytorch-fm/torchfm/model/test_fx/exp/model_repo/onn/onn_{batch}_{num_field}_{prefix}_{dim}_ori.onnx'
  modify_model_name = f'/home/yssun/pytorch-fm/torchfm/model/test_fx/exp/model_repo/onn_mlp/onn_{batch}_{num_field}_{prefix}_{dim}_modify.onnx'
  ori, modify = workload_onn(num_field,prefix,dim,l = [1024,512,256],batch = batch)
  # torch.onnx.export(ori,               # 模型 being run
  #                 torch.randint(low=0, high=20, size=(batch,num_field), dtype=torch.long),                  # 模型输入 (or a tuple for multiple inputs)
  #                 ori_model_name,        # 导出文件的文件名
  #                 export_params=True, # 如果设置为True，则参数也会被导出。注意某些情况下参数可能无法被导出。
  #                 opset_version=10,   # ONNX版本
  #                 do_constant_folding=True,  # 是否执行常量折叠以优化模型
  #                 input_names = ['input'],   # 输入的名称
  #                 output_names = ['output'], # 输出的名称
  #                 )
  torch.onnx.export(modify,               # 模型 being run
                  torch.randint(low=0, high=20, size=(batch,num_field), dtype=torch.long),                  # 模型输入 (or a tuple for multiple inputs)
                  modify_model_name,        # 导出文件的文件名
                  export_params=True, # 如果设置为True，则参数也会被导出。注意某些情况下参数可能无法被导出。
                  opset_version=10,   # ONNX版本
                  do_constant_folding=True,  # 是否执行常量折叠以优化模型
                  input_names = ['input'],   # 输入的名称
                  output_names = ['output'], # 输出的名称
                  )

In [30]:
dims= [32]
batches = [1024,2048,4096]
num_field_and_prefixs = [(34 ,29),(22 ,10)]

In [31]:
for dim in dims:
  for batch in batches:
    for num_field,prefix in num_field_and_prefixs:
      genWorkload(num_field=num_field,prefix=prefix,batch=batch,dim=dim)

now gen workload of ONN with config: dim: 32, num_field: 34, prefix: 29


/home/yssun/miniconda3/envs/tensorrt/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:1173: UserWarning: This model contains a squeeze operation on dimension 1. If the model is intended to be used with dynamic input shapes, please use opset version 11 to export the model.
  warnings.warn(


now gen workload of ONN with config: dim: 32, num_field: 22, prefix: 10


/home/yssun/miniconda3/envs/tensorrt/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:1173: UserWarning: This model contains a squeeze operation on dimension 1. If the model is intended to be used with dynamic input shapes, please use opset version 11 to export the model.
  warnings.warn(


now gen workload of ONN with config: dim: 32, num_field: 34, prefix: 29


...

/home/yssun/miniconda3/envs/tensorrt/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:1173: UserWarning: This model contains a squeeze operation on dimension 1. If the model is intended to be used with dynamic input shapes, please use opset version 11 to export the model.
  warnings.warn(


now gen workload of ONN with config: dim: 32, num_field: 34, prefix: 29


/home/yssun/miniconda3/envs/tensorrt/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:1173: UserWarning: This model contains a squeeze operation on dimension 1. If the model is intended to be used with dynamic input shapes, please use opset version 11 to export the model.
  warnings.warn(


now gen workload of ONN with config: dim: 32, num_field: 22, prefix: 10


/home/yssun/miniconda3/envs/tensorrt/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:1173: UserWarning: This model contains a squeeze operation on dimension 1. If the model is intended to be used with dynamic input shapes, please use opset version 11 to export the model.
  warnings.warn(
